<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
$\beta\beta0\nu$ Stuy </p><br>

## Study $\beta\beta0\nu$ reconstruction 

In [1]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

#### General

In [2]:
# General importings
import os
import sys
import glob
import logging
import math
import numpy  as np
import warnings
import functools
import time

from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D 
import networkx as nx
from itertools   import combinations

In [3]:
from pandas import DataFrame, Series
from typing import List, Tuple
from typing import Union
from   dataclasses import dataclass

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
plt.rcParams["figure.figsize"] = 8, 6
plt.rcParams["font.size"     ] = 14

In [6]:
idx = pd.IndexSlice

#### IC

In [7]:

import invisible_cities.core.system_of_units  as units

from invisible_cities.io.mcinfo_io import load_mcconfiguration
from invisible_cities.io.mcinfo_io import load_mcparticles_df
from invisible_cities.io.mcinfo_io import load_mchits_df
from invisible_cities.io.mcinfo_io import load_mcsensor_positions
from invisible_cities.io.mcinfo_io import load_mcsensor_response_df
from invisible_cities.io.mcinfo_io import get_sensor_types
from invisible_cities.io.mcinfo_io import get_sensor_binning
from invisible_cities.io.mcinfo_io import get_event_numbers_in_file
from invisible_cities.core.core_functions import in_range


#### TICs

In [8]:
from tics.histograms    import h1
from tics.pd_tics       import get_index_slice_from_multi_index
from tics.graphics_reco import plot_hits3d
from tics.stats_tics    import bin_data_with_equal_bin_size
from tics.util_tics import get_class_name
from tics.util_tics import Range
from tics.graphics_reco import plot_gtrack
from tics.system_of_units import *

#### NetFlex

In [9]:
from nextflex.core import Setup

In [10]:
from nextflex.core import Setup
from nextflex.mctrue_functions import get_mc_particles
from nextflex.mctrue_functions import get_mc_primary_particles
from nextflex.mctrue_functions import get_mc_vertex
from nextflex.mctrue_functions import select_mc_particles
from nextflex.mctrue_functions import get_mc_hits
from nextflex.mctrue_functions import select_mc_hits
from nextflex.mctrue_functions import total_hit_energy
from nextflex.mctrue_functions import get_event_hits_from_mchits

In [11]:
from nextflex.reco_functions import voxelize_hits
from nextflex.reco_functions import get_voxels_as_list
from nextflex.reco_functions import voxel_position
from nextflex.reco_functions import voxel_energy
from nextflex.reco_functions import voxel_nhits
from nextflex.reco_functions import distance_between_two_voxels
from nextflex.reco_functions import voxel_distances
from nextflex.reco_functions import voxel_distance_pairs
from nextflex.reco_functions import make_track_graphs
from nextflex.reco_functions import gtrack_voxels
from nextflex.reco_functions import shortest_paths
from nextflex.reco_functions import find_extrema_and_length
from nextflex.reco_functions import GTrack
from nextflex.reco_functions import voxels_in_blob
from nextflex.reco_functions import blob_energy
from nextflex.reco_functions import write_gtracks_json
from nextflex.reco_functions import load_gtracks_json

In [12]:
from nextflex.reco_analysis import reco_gtrack
from nextflex.reco_analysis import reco_gtrack
from nextflex.reco_analysis import reco_gtrack
from nextflex.reco_analysis import reco_gtrack_stats_histos
from nextflex.reco_analysis import reco_gtrack_timing_histos
from nextflex.reco_analysis import reco_gtrack_blobs
from nextflex.reco_analysis import reco_gtrack_blobs_histos
from nextflex.reco_analysis import TrackRecoStats
from nextflex.reco_analysis import TrackRecoTiming
from nextflex.reco_analysis import TrackRecoEventStats
from nextflex.reco_analysis import save_to_JSON
from nextflex.reco_analysis import load_from_JSON
from nextflex.reco_analysis import TrackRecoAnalysisSetup
from nextflex.reco_analysis import GtrkStats
from nextflex.reco_analysis import gtrack_summary

### Input data

In [13]:
FDATA = os.environ['FLEXDATA']

In [14]:
setup = Setup(flexDATA = FDATA,
              sipmPDE  = 1.0,
              maskPDE  = 1.0,
              qTh      = 0.0,
              tpConfig = "FLEX100_M6_O6_EL8bar_bb0nu")
print(setup)


        Setup <FLEX100_M6_O6_EL8bar_bb0nu_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0>:
        tracking plane configuration = FLEX100_M6_O6_EL8bar_bb0nu
        sipm PDE                     = 1.0
        transmission of teflon masks = 1.0
        charge threshold             = 0.0
        energy sensors               = PMTs
        pitch                        = 15.55
        number of energy sensors     = 60
        number of SiPMs              = 3093
        root directory               = /Users/jj/Development/flexdata/FLEX100_M6_O6_EL8bar_bb0nu
        analysis directory           = /Users/jj/Development/flexdata/analysis/FLEX100_M6_O6_EL8bar_bb0nu
        number of h5 files in dir    = 208
        sipm map at                  = /Users/jj/Development/flexdata/flexmaps/sipm_map_15.6_mm.csv
        sipm map name                = sipm_map_15.6_mm.csv
        output file                  = /Users/jj/Development/flexdata/kdsts/FLEX100_M6_O6_EL8bar_bb0nu_PMTs_sipmPDE_1.0_maskPDE_1.0_qTh_0.0.cs

In [15]:
ifnames = glob.glob(f"{FDATA}/{setup.tpConfig}/*.h5")
print(f'found {len(ifnames)} files')

found 208 files


### Run with large voxels

In [17]:
voxel_bin  = 5
contiguity =10

In [19]:
gtrksEvt, trackRecoStats, trackRecoEventStats, trackRecoTiming = reco_gtrack([ifnames[0]], 
                                                                          voxel_bin, 
                                                                          contiguity, 
                                                                          debug=False)

 Total events analyzed = 4,
               Events with a single track = 3


In [24]:
gtrks = gtrksEvt[0]

In [26]:
nv = [len(gt.voxels) for gt in gtrks ]
nv

[47, 1, 1, 1]

### Save to disk

In [ ]:
traSetup = TrackRecoAnalysisSetup(setup, voxel_bin, contiguity, gtrks,
                                 trackRecoStats, trackRecoEventStats, trackRecoTiming)

In [ ]:
traSetup

In [ ]:
save_to_JSON(traSetup.trackRecoStats, traSetup.fileTrackRecoStats, numpy_convert=True)

In [ ]:
save_to_JSON(traSetup.trackRecoTiming, traSetup.fileTrackRecoTiming, numpy_convert=True)

In [ ]:
save_to_JSON(traSetup.trackRecoEventStats, traSetup.fileTrackRecoEventStats, numpy_convert=False)

In [ ]:
write_gtracks_json(traSetup.gtracks, traSetup.fileGTracks)

### Load from disk

In [ ]:
trs  = TrackRecoStats(**load_from_JSON(traSetup.fileTrackRecoStats))

In [ ]:
trt  = TrackRecoTiming(**load_from_JSON(traSetup.fileTrackRecoTiming))

In [ ]:
tres = TrackRecoEventStats(**load_from_JSON(traSetup.fileTrackRecoEventStats))

In [ ]:
gtrks = load_gtracks_json(traSetup.fileGTracks)

### Run stats and timing

In [ ]:
tres

In [ ]:
reco_gtrack_stats_histos(trs, figsize=(14,10))

In [ ]:
reco_gtrack_timing_histos(trt, figsize=(14,10))

### Blobs

In [ ]:
gts = reco_gtrack_blobs(gtrks, rb = 5)

In [ ]:
reco_gtrack_blobs_histos(gts, figsize=(14,10))

In [ ]:
gts = reco_gtrack_blobs(gtrks, rb = 10)

In [ ]:
reco_gtrack_blobs_histos(gts, figsize=(14,10))

In [ ]:
gts = reco_gtrack_blobs(gtrks, rb = 15)

In [ ]:
reco_gtrack_blobs_histos(gts, figsize=(14,10))

In [ ]:
gts = reco_gtrack_blobs(gtrks, rb = 20)

In [ ]:
reco_gtrack_blobs_histos(gts, figsize=(14,10))

### Display a few events

In [ ]:
ts = reco_gtrack_blobs(gtrks, rb = 5)

In [ ]:
plot_gtrack(gtrks[0],
            rblob     = 5,
            autorange = True,
            weight    = 'energy',
            xrange    = (-500,500),
            yrange    = (-500,500),
            zrange    = (0,1000),
            figsize   = (14,10))

In [ ]:
gtrack_summary(gts,0)

In [ ]:
plot_gtrack(gtrks[1], rblob = 5, figsize = (14,10))

In [ ]:
gtrack_summary(gts,1)

In [ ]:
plot_gtrack(gtrks[2], rblob = 5, figsize = (14,10))

In [ ]:
gtrack_summary(gts,2)

In [ ]:
plot_gtrack(gtrks[5], rblob = 5, figsize = (14,10))

In [ ]:
gtrack_summary(gts,5)

In [ ]:
plot_gtrack(gtrks[10], rblob = 5, figsize = (14,10))

In [ ]:
gtrack_summary(gts,10)